# Install Libraries (if necessary)

In [9]:
# !pip install textblob
# !pip install tweepy

# Import Libraries

In [10]:
# Import Libraries
from textblob import TextBlob
import re
import sys
import tweepy
import matplotlib.pyplot as plt
import os
import string

import mysql.connector

# Twitter API Credentials

In [11]:
# Authentication credentials of Twitter API

consumerKey = "XXXXXXXXXX"
consumerSecret = "XXXXXXXXXXXXXXXXXXXXXXXXX"
accessToken = "XXXXXXXXXXXXXXXXXXXXX"
accessTokenSecret = "XXXXXXXXXXXXXXXXXXXXXXXXXXX"

auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

# Create and Connect to Database

In [12]:
# Connect to MySQL

db = mysql.connector.connect(host='localhost', database='TwitterAnalysis', user='root', password='')
if db.is_connected():
    print("CONNECTED TO MYSQL DATABASE! \n.\n.\n.\n.")
cur = db.cursor(buffered = True)


# Check if Table already exists. If not, create the Table in our Database

q1 = "SHOW TABLES LIKE 'TweetStorage'" 
cur.execute(q1)
res = cur.fetchall()
if res:
    print('This table already exist')
else:
    table_creation = "CREATE TABLE TweetStorage (id int AUTO_INCREMENT, Username varchar(30), Followers int, Num_Tweets int, Num_Retweets int, Tweet_Text varchar(400), Create_Date varchar(30),Likes int, Location varchar(50), Hashtags varchar(400), Sentiment varchar(15), primary key (id));"
    cur.execute(table_creation)
    db.commit()
    print("Table created successfully!")
    cur.execute("SHOW TABLES")

CONNECTED TO MYSQL DATABASE! 
.
.
.
.
This table already exist


### Clean Tweets Function

In [13]:
# Function for cleaning the text from emojissymbols & URLs

def clean_text(txt):
    emoji_pat = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\ufe0f"  # dingbats
                               "]+", flags=re.UNICODE)
      
    # Clean text from Emojis
    clean_text = emoji_pat.sub(r'', txt)
    
    
    # Clean Text from URL
    clean_text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', clean_text)
    
    
    # Clean Text from Mentions
    clean_text = re.sub("@[A-Za-z0-9_]+","", clean_text)

    return clean_text

### TextBlob - Determine Polarity Functon

In [14]:
# Function for setting the polarity to Positive / Negative / Neutral using TextBlob

def sentiment_pol(cl_text):
    if TextBlob(cl_text).sentiment.polarity > 0.05:
        polarity = 'Positive'
    elif TextBlob(cl_text).sentiment.polarity < -0.05:
        polarity = 'Negative'
    else:
        polarity = 'Neutral'
    return polarity

# Fetching Tweets and Upload to Database Function 

In [15]:
def TweetRequest(word,num):

    keyword = word
    noOfTweet = num
    
    # Variables to determine how much of the fetched tweets are actually written in the database.
    # Duplicate tweets will be discarted
    records_created = 0
    duplicates = 0 
    
    # Initializing tweepy. Excluding retweets.
    tweets = tweepy.Cursor(api.search_tweets, tweet_mode="extended", q=keyword + ' -filter:retweets',lang='en').items(noOfTweet) # Connect with API and make request
    
    

    for tweet in tweets:
        
        cl_text = clean_text(tweet.full_text)   # clean text using the clean_text function above
            
        User = tweet.user.screen_name,
        Followers = tweet.user.followers_count,
        Number_Tweets = tweet.user.statuses_count,
        Number_Retweets = tweet.retweet_count,
        Tweet_Creation_Date = str(tweet.created_at),
        Likes = tweet.favorite_count,
        Location = tweet.user.location,
        Hashtag_list = re.findall(r"#(\w+)", tweet.full_text)  # finding all hashtags in the tweet
        Hashtags = ' '.join([str(i) for i in Hashtag_list]) # put all hashtag words of the tweet in a single string
        Polarity = sentiment_pol(cl_text) 
            
            
        # Transfer the data into the database
            
        write_to_db = "INSERT INTO TweetStorage (Username, Followers, Num_Tweets, Num_Retweets, Tweet_Text, Create_Date, Likes, Location, Hashtags, Sentiment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
            
        # Values to be inserted on the record    
        insert_values = (
            User[0],
            Followers[0],
            Number_Tweets[0],
            Number_Retweets[0],
            cl_text,
            Tweet_Creation_Date[0][:16], 
            Likes[0],
            Location[0],
            Hashtags,
            Polarity)
        
        
        #Point on the Table to get a all the rows of the table as a list of tupples, in order to check for duplicates.
        
        cur.execute("SELECT Username,Followers,Num_Tweets,Num_Retweets,Tweet_Text,Create_Date,Likes,Location,Hashtags,Sentiment FROM TweetStorage")
        table_db = cur.fetchall()
        
                
        # Check if there are duplicate posts of the same user and skip them from importing to Table
        
        if insert_values not in table_db:
            cur.execute(write_to_db, insert_values)
            db.commit()
            records_created += 1
        else:
            duplicates += 1
            
    print(noOfTweet, " Records Requested.")        
    print(records_created, " Records Created.")
    print(duplicates, "Duplicates Found")

# Run The TweetRequest Function

## Liverpool

In [16]:
TweetRequest("#LiverpoolFC",1000)

1000  Records Requested.
994  Records Created.
6 Duplicates Found


## Mancehster United

In [17]:
TweetRequest("#ManUtd",1000)

1000  Records Requested.
998  Records Created.
2 Duplicates Found


## Manchester City

In [7]:
TweetRequest("#ManCity",1000)

1000  Records Requested.
994  Records Created.
6 Duplicates Found
